# LABORATORIO #1 
 Autores: Alfredo Quezada
 
 Autores: Randy Venegas

### Parte 1

* 1. Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones. 

In [3]:
import pandas as pd
from urllib.parse import urlparse

df = pd.read_csv('dataset_pishing.csv')
df.sample(n=5).head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
9931,https://docs.google.com/document/d/e/2PACX-1vQ...,0,0,0,0,0,0,0,0,0,...,1,1,0,2974,8348,1,0,1,10,phishing
3746,https://lingust.ru:443/,0,0,0,0,0,0,1,0,0,...,1,1,0,358,3659,13900,0,1,4,legitimate
4599,https://kenalice.tw/,0,0,0,0,0,0,0,0,0,...,0,0,0,696,1130,68131,0,0,2,legitimate
9228,http://www.drucker.com.ar/inc/aaa.htm,0,1,0,0,0,0,0,0,0,...,1,0,0,170,-1,0,0,1,0,phishing
9710,https://www.google.nu/?gws_rd=ssl,0,1,0,0,0,0,0,0,0,...,0,0,0,319,7717,891900,0,0,4,legitimate


* 2. Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como 
“pishing”. ¿Está balanceado el dataset? 

In [4]:
print(df['status'].value_counts())

legitimate    5715
phishing      5715
Name: status, dtype: int64


        El modelo, como se puede observar si se encuentra balanceado. 

* 3. Para ello escriba las funciones necesarias y genere las nuevas columnas del dataset. Muestre un nuevo 
ejemplo de cinco observaciones donde se visualicen algunas de las columnas nuevas. 

In [5]:
#In the first function, the goal is to find the length of the domain of the page
import re

def domain_length(url):
    parsed_uri = urlparse(url)
    domain_name = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    return len(domain_name)


df_with_features = df.assign(domain_length = df['url'].apply(domain_length))



#In the second function, special characters are counted
def count_special_characters(url):
    if re.search(r'[.,@/?&|=~#%$:*;,! ]', url):
        return 0
    else:
        return 1

df_with_features['has_special_characters'] = df_with_features['url'].apply(count_special_characters)


#The third function to see the digits in the domain names
def count_digits_in_domain(url):
    return len(re.findall(r'\d', urlparse(url).netloc))
df_with_features['digits_in_domain'] = df_with_features['url'].apply(count_digits_in_domain)

#The fourth function is to have the https in the url
def has_https(url):
    if 'https' in url:
        return 1
    else:
        return 0
df_with_features['has_https'] = df_with_features['url'].apply(has_https)

#The fifth function is to obtain the digits in the url
def count_digits(url):
    return len(re.findall(r'\d', url))
df_with_features['digits_count'] = df_with_features['url'].apply(count_digits)

df_with_features.sample(n=5).head()



,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,web_traffic,dns_record,google_index,page_rank,status,domain_length,has_special_characters,digits_in_domain,has_https,digits_count
7669,http://www.clker.com/clipart-computer-mouse-1....,0,1,1,0,0,0,0,1,0,...,56709,0,0,5,legitimate,21,0,0,0,1
3104,http://www.driverscape.com/download/usb-to-par...,0,1,0,0,0,0,0,0,0,...,5219,0,1,4,legitimate,27,0,0,0,5
2284,https://math.stackexchange.com/questions/50296...,0,0,0,0,0,0,0,0,0,...,98,0,0,5,legitimate,31,0,0,1,6
6854,http://www.videoskick.com/acrobat/error.php,0,1,0,0,0,0,0,0,0,...,0,0,1,2,phishing,26,0,0,0,0
9750,https://betasus020.blogspot.com,0,0,0,0,0,0,0,0,0,...,0,0,1,5,phishing,32,0,3,1,3


Cambiar la variable a binaria 

In [6]:
df_with_features['status'] = df_with_features['status'].map({'legitimate': 1, 'phishing': 0})
df_with_features = df_with_features.drop('url', axis=1)
df_with_features.head()

,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,...,web_traffic,dns_record,google_index,page_rank,status,domain_length,has_special_characters,digits_in_domain,has_https,digits_count
0,0,1,0,0,0,0,0,0,0,0,...,0,1,1,4,1,27,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,2,0,31,0,0,0,17
2,1,0,1,0,0,0,0,0,1,0,...,5828815,0,1,0,0,59,0,0,1,19
3,0,0,0,0,0,0,0,0,0,0,...,107721,0,0,3,1,19,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,8725,0,0,6,1,23,0,0,0,0


Visualizacion


In [7]:
# Se llama a la libreria y luego creamos el archivo html. 
import pandas_profiling
pandas_profiling.ProfileReport(df_with_features).to_file('reportF.html')

# Determinar la varianza con la columna status
df_with_features.corr()['status'].sort_values(ascending=False).head(10)


Summarize dataset:  92%|█████████▏| 72/78 [00:05<00:00, 31.52it/s, Calculate auto correlation]                  C:\Users\Quezada\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\multimethod\__init__.py:315: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  return func(*args, **kwargs)
Export report to file: 100%|██████████| 1/1 [00:00<00:00,  2.99it/s]


status                   1.000000
page_rank                0.511137
nb_www                   0.443468
nb_hyperlinks            0.342628
domain_age               0.331889
ratio_intHyperlinks      0.243982
ratio_intMedia           0.193333
links_in_tags            0.184401
safe_anchor              0.173397
domain_with_copyright    0.173098
Name: status, dtype: float64

In [9]:
# Ahora borramos los irrelevantes que encontramos en el reporte
df2 = df_with_features[['google_index', 'domain_in_title', 'phish_hints', 'nb_www', 'nb_hyperlinks', 'domain_age', 'page_rank', 'ratio_intHyperlinks', 'status', 'prefix_suffix', 'longest_word_path']]
df2.head()

,google_index,domain_in_title,phish_hints,nb_www,nb_hyperlinks,domain_age,page_rank,ratio_intHyperlinks,status,prefix_suffix,longest_word_path
0,1,0,0,1,17,-1,4,0.529412,1,0,6
1,1,1,0,0,30,5767,2,0.966667,0,0,32
2,1,1,0,0,4,4004,0,1.000000,0,1,17
3,0,1,0,0,149,-1,3,0.973154,1,0,0
4,0,0,0,1,102,8175,6,0.470588,1,0,11
